# Task 3

In [0]:
import pandas as pd
import io
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
size = "small"

In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 


In [0]:
def func(x):
  L = []
  for i in x.index:
    L.append(str(x.loc[i]['manufacturer'])+' '+str(x.loc[i]['model']))
  return L

In [0]:
def task_3(spark_session, flights_path, airlines_path, aircrafts_path, country):
  flights_df = spark_session.read.csv(flights_path,
                                        inferSchema=True,
                                        header=True)
  flights_df = clean_column_names(flights_df).dropna()
  
  airlines_df = spark_session.read.csv(airlines_path,
                                        inferSchema=True,
                                        header=True)
  airlines_df = clean_column_names(airlines_df).dropna()
  
  aircrafts_df = spark_session.read.csv(aircrafts_path,
                                        inferSchema=True,
                                        header=True)
  aircrafts_df = clean_column_names(aircrafts_df).dropna()
  
  # select country
  airlines_df = airlines_df.filter(airlines_df.country==country)
  # select flights
  flights_df = flights_df.select(flights_df.carrier_code,flights_df.tail_number)
  # select aricrafts
  aircrafts_df = aircrafts_df.select(aircrafts_df.tailnum,aircrafts_df.manufacturer,aircrafts_df.model)

  join_data = airlines_df.join(flights_df, airlines_df.carrier_code == flights_df.carrier_code, "left")
  join_data = join_data.join(aircrafts_df, join_data.tail_number == aircrafts_df.tailnum, "left")
  join_data = join_data.select(airlines_df.name,join_data.manufacturer,join_data.model).dropna()
  join_data = join_data.groupBy(join_data.name,join_data.manufacturer,join_data.model).count()
  join_data = join_data.sort(join_data.name,join_data['count'].desc())
  join_data = join_data.select(join_data.name,join_data.manufacturer,join_data.model)
  data = join_data.toPandas()
  data = data.groupby('name').head(5)
  result = data.groupby('name').apply(func).head(5)
  
  
  res_str = result.to_csv().replace('\n', ';')
  resDf = pd.read_csv(io.StringIO(res_str), lineterminator=';')
  resDf['combined'] = resDf['name'] + ' \t ' +  resDf['0']
  
  resDf = resDf['combined']
  resDf = resDf.to_frame()
#   resDf.coalesce(1).write.mode("overwrite").csv("dbfs:/FileStore/tables/task1")
  display(resDf)
  

In [0]:
task_3(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")

combined
"AirTran ['BOEING 717-200', 'BOEING 737-7BD', 'BOEING 737-76N']"
"Alaska Airlines Inc. ['BOEING 737-4Q8', 'BOEING 737-490', 'BOEING 737-890', 'BOEING 737-790', 'BOEING 737-990']"
"American Airlines Inc. ['MCDONNELL DOUGLAS DC-9-82(MD-82)', 'MCDONNELL DOUGLAS DC-9-83(MD-83)', 'BOEING 757-223', 'AIRBUS INDUSTRIE A319-112', 'AIRBUS INDUSTRIE A319-132']"
"American Eagle Airlines Inc. ['EMBRAER EMB-145LR', 'EMBRAER EMB-135KL', 'EMBRAER EMB-135LR', 'BOMBARDIER INC CL-600-2C10', 'SAAB-SCANIA SAAB 340B']"
"Atlantic Southeast Airlines ['BOMBARDIER INC CL-600-2B19', 'EMBRAER EMB-145LR', 'EMBRAER EMB-145XR', 'BOMBARDIER INC CL-600-2C10', 'CANADAIR CL-600-2B19']"
